In [ ]:
import sys
import boto3
import tensorflow as tf
import numpy as np
from PIL import Image
import io

# S3 Configuration
s3 = boto3.client('s3')

# Arguments passed to the script
input_image_s3_path = sys.argv[1]  
result_bucket_name = sys.argv[2]  

# Load trained model
model_bucket = 'my-models-bucket'
model_key = 'trained_model.h5'
local_model_path = '/tmp/trained_model.h5'

s3.download_file(model_bucket, model_key, local_model_path)
model = tf.keras.models.load_model(local_model_path)

# Preprocess image
def preprocess_image(image_s3_path):
    bucket_name, key = image_s3_path.replace("s3://", "").split("/", 1)
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    img = Image.open(io.BytesIO(obj['Body'].read())).resize((224, 224))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Predict image
def predict(image_s3_path):
    image = preprocess_image(image_s3_path)
    prediction = model.predict(image)
    if prediction[0][0] > 0.5:
        return "Cancer Detected"
    else:
        return "No Cancer Detected"

# Save prediction to S3
def save_prediction(result_bucket, image_s3_path, prediction):
    result_key = f"{image_s3_path.split('/')[-1].split('.')[0]}_result.txt"
    s3.put_object(Bucket=result_bucket, Key=result_key, Body=prediction)

# Main execution
if __name__ == "__main__":
    prediction = predict(input_image_s3_path)
    save_prediction(result_bucket_name, input_image_s3_path, prediction)
